In [9]:
import io, os, sys, types
from IPython import get_ipython
from Utils import NotebookFinder
import pandas as pd
from Attack_RF import attack_RF
from config_fileRF import config
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt  
import numpy as np
from Data_preprocessing import procedata
import warnings
from sklearn.decomposition import PCA
import time

In [10]:
n_components = [81]#list(range(2,81,3))
n_splits_k =  config["RF"]["n_splits_k"]

In [11]:
def run_pca():
    datasets = pd.read_csv("data/attack_RF.csv")
    print (datasets.shape) 
    features = datasets.iloc[:, :-1]
    labels = datasets.iloc[:, -1]
    skf = StratifiedKFold(n_splits=n_splits_k,random_state = 2)
    # The accracy_of_classes list holds the accuracy of each compromise
    accracy_of_classes = []
    # The precision_of_classes list holds the precision of each class in each compromise test set
    precision_of_classes = []
    # The recall_of_classes list holds the recall value of each class in each compromise test set
    recall_of_classes = []
    # The f1_of_classes list holds the f1 value of each class in each compromise test set
    f1_of_classes = []
    # flag is used to mark whether to output the training set and test set size，
    # flag = 0 means no output, flag = 1 means output
    flag = 0 
    accuracy_mean = []
    length_components = len(n_components)
    components_flag = 0
    precision_com =  np.zeros((14,length_components))
    recall_com= np.zeros((14,length_components))
    f1_com= np.zeros((14,length_components))
    for n_com in n_components:
        print("pca compressed dimension is：",n_com)
        #estimator = PCA(n_components=n_com)
        for train_index, test_index in skf.split(features,labels):  
            start = time.clock()
            print("a")
            # The following is the training and test set segmentation process
            features_train = features.iloc[train_index]
            features_test =  features.iloc[test_index]
            labels_train = labels.iloc[train_index]
            labels_test =  labels.iloc[test_index]
            features_train = features_train.values
            labels_train = labels_train.values
            features_test = features_test.values
            labels_test = labels_test.values
            labels_train = labels_train.reshape(len(labels_train))
            labels_test = labels_test.reshape(len(labels_test))
            if flag==0:
                print("Number of examples in the train set:",labels_train.shape)
                print("Number of examples in the test set:",labels_test.shape)
                flag = 1
            # attack_RF returns the accuracy of each test set and the precision, recall, and f1 values of each class 
#             features_train_pca = estimator.fit_transform(features_train)
#             features_test_pca = estimator.transform(features_test)
            accuracy,precision,recall,f1 = attack_RF(features_train,labels_train,features_test,labels_test)
            precision_of_classes.append(precision)
            recall_of_classes.append(recall)
            f1_of_classes.append(f1)
            accracy_of_classes.append(accuracy)
        
        end = time.clock()
        print("Running time: %s seconds"%(end - start))
        # Find the average of all the precisions in all prediction sets in each compromise
        precision_mean=np.mean(precision_of_classes,axis=0)
        # Find the average of all the recall value in all prediction sets in each compromise
        recall_mean = np.mean( recall_of_classes,axis=0)
        # Find the average of all the f1 value in all prediction sets in each compromise
        f1_mean = np.mean(f1_of_classes,axis=0)
        acc_avg = np.mean(accracy_of_classes)
        accuracy_mean.append(acc_avg)
        for i in range(14):
            precision_com[i][components_flag] = precision_mean[i]
            recall_com[i][components_flag] = recall_mean[i]
            f1_com[i][components_flag] = f1_mean[i]
        print("accuracy_mean value is :",acc_avg)
        # Visualize the precision average, recall average, and f1 average of all classes in each compromise.
        plt.figure()
        x = range(1,len(precision)+1)
        y1 = precision_mean
        y2 = recall_mean
        y3 = f1_mean
        pre = plt.plot(x,y1,label = "precision")
        rec = plt.plot(x,y2,label = "recall")
        f1_ = plt.plot(x,y3,label = "f1") 
        plt.xlabel('labels of attack')
        plt.legend(['precision','recall','f1'])
        plt.show()
        
        plt.figure()
        x = n_components[0:components_flag+1]
        y1 = accuracy_mean
        acc_mean = plt.plot(x,y1,label = "accuracy_mean")   
        plt.xlabel('n_component')
        plt.ylabel('accuracy mean')
        plt.show()
        
        for i in range(14):   
            plt.figure()
            x = n_components[0:components_flag+1]
            y1 = precision_com[i][0:components_flag+1]
            y2 = recall_com[i][0:components_flag+1]
            y3 = f1_com[i][0:components_flag+1]
            pre = plt.plot(x,y1,label = "precision of label{}".format(i))
            rec = plt.plot(x,y2,label = "recall of label{}".format(i))
            f1_ = plt.plot(x,y3,label = "f1 of label{}".format(i)) 
            plt.xlabel('n_component')
            plt.legend(["precision of label{}".format(i),"recall of label{}".format(i),"f1 of label{}".format(i)])
            plt.show()
        components_flag +=1

In [ ]:
if __name__ == "__main__":
    run_pca()

(1252936, 82)
pca compressed dimension is： 81
a
Number of examples in the train set: (1127635,)
Number of examples in the test set: (125301,)
a
a
a
a
a
a
a
